In [ ]:
model_dir = "./models/optimized/stable-diffusion-v1-5/stable-diffusion-v1-5"

ExecutionProvider="QNNExecutionProvider"

In [ ]:
# reference: https://learn.microsoft.com/en-us/windows/ai/new-windows-ml/tutorial?tabs=python#acquiring-the-model-and-preprocessing
import subprocess
import json
import sys
import os
import onnxruntime as ort

def register_execution_providers():
    worker_script = os.path.abspath('winml.py')
    print(worker_script)
    result = subprocess.check_output([sys.executable, worker_script], text=True)
    paths = json.loads(result)
    for item in paths.items():
        try:
            ort.register_execution_provider_library(item[0], item[1])
        except Exception as e:
            print(f"Failed to register execution provider {item[0]}: {e}")

register_execution_providers()

In [ ]:
import sys
from pathlib import Path

NOTEBOOK_DIR = Path(__file__).parent if "__file__" in globals() else Path.cwd()
PROJECT_ROOT = NOTEBOOK_DIR.parents[1]
sys.path.insert(0, str(PROJECT_ROOT))

import numpy as np
from sd_utils.qdq import OnnxStableDiffusionPipelineWithSave

def add_ep_for_device(session_options, ep_name, device_type, ep_options=None):
    ep_devices = ort.get_ep_devices()
    for ep_device in ep_devices:
        if ep_device.ep_name == ep_name and ep_device.device.type == device_type:
            print(f"Adding {ep_name} for {device_type}")
            session_options.add_provider_for_devices([ep_device], {} if ep_options is None else ep_options)
            break


session_options = ort.SessionOptions()
provider_options = [{}]

add_ep_for_device(session_options, ExecutionProvider, ort.OrtHardwareDeviceType.NPU)

pipeline = OnnxStableDiffusionPipelineWithSave.from_pretrained(
    model_dir, provider=ExecutionProvider, sess_options=session_options, provider_options=provider_options
)
pipeline.save_data_dir = None

prompt = "A beautiful landscape painting of mountains during sunrise"
result = pipeline(
    [prompt],
    num_inference_steps=25,
    height=512,
    width=512,
    guidance_scale=7.5,
    generator=np.random.RandomState(seed=0)
)

from IPython.display import display
display(result.images[0])
